In [1]:
import pandas as pd
import plotly.express as px
import datetime
import matplotlib.pyplot as plt

In [2]:
# Read the Excel file into a dictionary of data frames, with sheet names as keys
xls = pd.read_excel('data/TurningMovementCounts.xlsx', sheet_name=None)
xls

{'7800':          Time  EBL  EBT  EBR  SBL  SBT
 0    00:00:00    2    5    2   12    7
 1    00:15:00    2    4    0    9    7
 2    00:30:00    3    1    1    0    3
 3    00:45:00    0    1    1    3    5
 4    01:00:00    4    6    0    9    5
 ..        ...  ...  ...  ...  ...  ...
 955  22:45:00    8    8    1   14   35
 956  23:00:00   12    5    3    8   10
 957  23:15:00    6    1    1   17   14
 958  23:30:00    5    6    2    9   13
 959  23:45:00    4    6    1    3   18
 
 [960 rows x 6 columns],
 '7801':          Time  EBL  EBT  EBR  WBL  WBT  WBR  NBL  NBT  NBR  SBL  SBT  SBR
 0    00:00:00    2    1    0    1    0    0    1    5    0    0   20    7
 1    00:15:00    2    0    0    0    0    0    2    5    0    0   22    5
 2    00:30:00    2    0    0    0    0    0    0    8    0    0   11   10
 3    00:45:00    1    0    0    2    1    0    0    4    0    0   16    3
 4    01:00:00    1    1    0    0    0    0    0    2    0    0   17    4
 ..        ...  ...  ...  .

In [3]:
# Initialize an empty list to store modified data frames
dfs = []

# Iterate through each sheet
for sheet_name, df in xls.items():
    # Add a new column "ID" with the sheet name
    df['ID'] = sheet_name
    # Add the data frame to the list
    dfs.append(df)


In [4]:
# Concatenate all data frames along columns, filling missing values with NA
result = pd.concat(dfs, ignore_index=True)



In [5]:
longdata = result.drop_duplicates().\
    melt(id_vars=['ID', 'Time'], var_name='Movement', value_name='Volume')


In [6]:
# Find out how many rows contain zeros
rows_with_zeros = (longdata == 0).sum(axis=1)
print("Number of rows with zeros:", (rows_with_zeros > 0).sum())

Number of rows with zeros: 19396


In [7]:
# Define a function to determine the value of the "NBSB" column
def determine_direction(Movement	):
    if 'N' in Movement	:
        return 'northbound'
    elif 'S' in Movement	:
        return 'southbound'
    else:
        return 'other'

In [8]:
longdata['NBSB'] = longdata['Movement'].apply(determine_direction)



In [9]:
# Remove rows with zeros
df_without_zeros = longdata[(longdata != 0).all(axis=1)]


In [10]:
# mean here because it is an average of multiple days in the data. Average per 15 min bin
cleandata = df_without_zeros.groupby(['ID', 'Time', 'Movement', 'NBSB'])['Volume'].\
    mean().\
    reset_index().\
    dropna(subset=['Volume']).\
    groupby(['ID', 'Time', 'NBSB'])['Volume'].\
    sum().\
    reset_index().\
    dropna(subset=['Volume'])
cleandata

,ID,Time,NBSB,Volume
0,7800,00:00:00,other,10.333333
1,7800,00:00:00,southbound,18.600000
2,7800,00:15:00,other,10.411111
3,7800,00:15:00,southbound,18.300000
4,7800,00:30:00,other,10.800000
...,...,...,...,...
3834,7819,23:30:00,other,10.251190
3835,7819,23:30:00,southbound,25.611111
3836,7819,23:45:00,northbound,14.708333
3837,7819,23:45:00,other,8.430556


In [11]:
am_data = cleandata[(cleandata['Time'] >= datetime.time(6, 0)) & (cleandata['Time'] <= datetime.time(9, 0))]
md_data = cleandata[(cleandata['Time'] >= datetime.time(9, 0)) & (cleandata['Time'] <= datetime.time(15, 0))]
pm_data = cleandata[(cleandata['Time'] >= datetime.time(15, 0)) & (cleandata['Time'] <= datetime.time(19, 0))]

In [12]:
am_data.groupby(['ID', 'NBSB'])['Volume'].\
    sum().\
    reset_index().\
    pivot(index='ID', columns='NBSB', values='Volume').\
    drop(columns='other')

NBSB,northbound,southbound
ID,,
7800,NaN,966.300000
7801,3783.263889,1860.531746
7802,5029.333333,2019.601984
7803,2995.047619,1608.833333
7804,3851.611111,2096.517857
7805,4069.222222,2590.305556
7811,3673.166667,2081.333333
7812,2586.444444,NaN
7813,3893.222222,1966.294444


In [13]:
# Sort data by Time
am_data.sort_values(by='Time', inplace=True)

# Group by ID and Time, sum the volumes
grouped_data = am_data.groupby(['ID', 'Time'])['Volume'].sum().reset_index()

# Plotting
fig = px.line(grouped_data, x='Time', y='Volume', color='ID', labels={'Volume': 'Volume', 'Time': 'Time'})

# Show the plot
fig.show()

C:\Users\nlant\AppData\Local\Temp\ipykernel_30136\226943624.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  am_data.sort_values(by='Time', inplace=True)


In [ ]:
# cumulative data
# Group by ID and Time, sum the volumes
cumulative_data = am_data.groupby(['ID', 'Time'])['Volume'].sum().groupby(level=0).cumsum().reset_index()

# Plotting
fig = px.line(cumulative_data, x='Time', y='Volume', color='ID', labels={'Volume': 'Volume', 'Time': 'Time'})

# Show the plot
fig.show()

In [ ]:
md_data.groupby(['ID', 'NBSB'])['Volume'].\
    sum().\
    reset_index().\
    pivot(index='ID', columns='NBSB', values='Volume').\
    drop(columns='other')

In [ ]:
pm_data.groupby(['ID', 'NBSB'])['Volume'].\
    sum().\
    reset_index().\
    pivot(index='ID', columns='NBSB', values='Volume').\
    drop(columns='other')

In [15]:
# Sort data by Time
pm_data.sort_values(by='Time', inplace=True)

# Group by ID and Time, sum the volumes
grouped_data = pm_data.groupby(['ID', 'Time'])['Volume'].sum().reset_index()

# Plotting
fig = px.line(grouped_data, x='Time', y='Volume', color='ID', labels={'Volume': 'Volume', 'Time': 'Time'})

# Show the plot
fig.show()

C:\Users\nlant\AppData\Local\Temp\ipykernel_30136\1754315918.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [21]:
cleandata

,ID,Time,NBSB,Volume
0,7800,00:00:00,other,10.333333
1,7800,00:00:00,southbound,18.600000
2,7800,00:15:00,other,10.411111
3,7800,00:15:00,southbound,18.300000
4,7800,00:30:00,other,10.800000
...,...,...,...,...
3834,7819,23:30:00,other,10.251190
3835,7819,23:30:00,southbound,25.611111
3836,7819,23:45:00,northbound,14.708333
3837,7819,23:45:00,other,8.430556


In [27]:
# I want to see the volumes over time
nbsb = cleandata[cleandata['NBSB'] != 'other'].groupby(['Time', 'ID'])['Volume'].sum().reset_index()
# Plotting
fig = px.line(nbsb, x='Time', y='Volume', color='ID',  title='Volume by Time of Day - 15 min Volumes')
fig.update_xaxes(title_text='Time')
fig.update_yaxes(title_text='Volume')
fig.show()